[//]: # ( Finnish Horticultural Weather Analysis )
[//]: # ( License: MIT License )
[//]: # ( Repository: https://github.com/outobecca/botanical-colabs )

# 🇫🇮 Finnish Horticultural Weather Analysis
**Version 1.0** | Created: 2025-11-04 | Suomeksi / In Finnish

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/outobecca/botanical-colabs/blob/main/notebooks/finnish_weather_analysis.ipynb)

## 📋 Yleiskatsaus / Overview

**Tarkoitus / Purpose:** Analysoi Suomen ilmastotietoja puutarhatuotannon näkökulmasta käyttäen FMI:n (Ilmatieteen laitos) avointa dataa.

**Purpose:** Analyze Finnish weather data for horticultural applications using FMI (Finnish Meteorological Institute) open data.

### 🎯 Käyttötapaukset / Use Cases
- 🌡️ Lämpötila-analyysi kasvukaudelle / Growing season temperature analysis
- 💧 Sadanta ja kastelutarpeen arviointi / Precipitation and irrigation needs
- ❄️ Hallariski ja suoja-aika / Frost risk and protection timing
- ☀️ Auringonpaiste ja valoanalyysi / Sunshine and light analysis
- 📊 Kasvukauden pituuden seuranta / Growing season length tracking
- 🌱 Optimaalinen kylvöaika / Optimal planting time

### 📡 Tietolähteet / Data Sources

| Lähde / Source | Tyyppi / Type | Kuvaus / Description |
|----------------|---------------|---------------------|
| **FMI Open Data** | Avoin / Open | Sääasemat, historialliset tiedot / Weather stations, historical data |
| **FMI API** | REST API | Reaaliaikainen data / Real-time data |
| **Sample Data** | Esimerkki / Example | Testidataa / Test data |

### 🔑 API-avain / API Key

FMI Open Data requires API key (free):
- 📝 Register at: https://ilmatieteenlaitos.fi/rekisteroityminen-avoimen-datan-kayttajaksi
- 🔑 Store in Colab Secrets as `FMI_API_KEY`

### ⚠️ Huomautukset / Notes
- FMI data is free for non-commercial use
- Weather station locations across Finland
- Historical data available from 1959
- Real-time updates every hour


## 📚 Tausta ja menetelmät / Background & Methodology

### Suomen ilmasto puutarhataloudessa / Finnish Climate in Horticulture

Suomen ilmasto asettaa erityisiä haasteita puutarhatuotannolle:
- **Lyhyt kasvukausi** - Growing season typically May-September
- **Hallariski** - Frost risk even in summer months
- **Vaihtelevia sääoloja** - Variable weather conditions
- **Vähän aurinkoa talvella** - Limited sunlight in winter

Finnish climate presents unique challenges for horticulture:
- Short growing season (May-September)
- Frost risk throughout the year
- High variability in weather
- Extreme seasonal light differences

### FMI Open Data API

FMI (Ilmatieteen laitos) tarjoaa kattavaa säädataa / FMI provides comprehensive weather data:
- 150+ weather stations across Finland
- Temperature, precipitation, wind, snow depth
- Historical data from 1959
- Hourly and daily observations
- Forecasts and warnings

### Analyysi / Analysis Methods
1. Growing Degree Days (GDD) calculation
2. Frost probability analysis
3. Precipitation patterns
4. Optimal planting windows
5. Climate trends over time


## ⚙️ Vaihe 1: Asennus / Step 1: Installation


In [ ]:
# ============================================================================# Library Installation and Import# ============================================================================"""Install required libraries for FMI data access and analysis.Asennetaan tarvittavat kirjastot FMI-datan käyttöön."""!pip install -q pandas numpy matplotlib seaborn requests xml.etree.ElementTree ipywidgets# Importsfrom typing import Dict, Optional, List, Any, Tuplefrom IPython.display import display, Markdown, HTMLimport pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsimport requestsimport xml.etree.ElementTree as ETfrom datetime import datetime, timedeltaimport ipywidgets as widgets# Colab secrets (for API key)try:    from google.colab import userdata    IN_COLAB = Trueexcept ImportError:    IN_COLAB = False# Stylingsns.set_style("whitegrid")plt.rcParams['figure.figsize'] = (14, 6)print("✅ Kirjastot asennettu / Libraries installed")

## 🔧 Vaihe 2: Asetukset / Step 2: Configuration

Valitse sääasema ja ajanjakso / Select weather station and time period.


In [ ]:
# ============================================================================# Configuration and API Setup# ============================================================================# FMI API KeyFMI_API_KEY = ""if IN_COLAB:    try:        FMI_API_KEY = userdata.get('FMI_API_KEY')        print("✅ FMI API key loaded from Colab Secrets")    except:        print("⚠️ FMI API key not found in Colab Secrets")        print("   Register at: https://ilmatieteenlaitos.fi/rekisteroityminen-avoimen-datan-kayttajaksi")else:    print("ℹ️ Running locally - set FMI_API_KEY manually or use sample data")# Weather station selection (FORM)print("\n🌡️ VALITSE SÄÄASEMA / SELECT WEATHER STATION:")print("  [1] Helsinki Kaisaniemi")print("  [2] Jokioinen (Horticultural research station)")print("  [3] Turku")print("  [4] Tampere")print("  [5] Jyväskylä")print("  [6] Oulu")print("  [7] Rovaniemi")print("  [8] Use sample data")station_choice = input("Valinta / Choice (1-8): ").strip() or '2'# Station mapping (FMISID codes)stations = {    '1': {'name': 'Helsinki Kaisaniemi', 'fmisid': '100971', 'lat': 60.18, 'lon': 24.94},    '2': {'name': 'Jokioinen', 'fmisid': '101104', 'lat': 60.81, 'lon': 23.50},    '3': {'name': 'Turku', 'fmisid': '100949', 'lat': 60.52, 'lon': 22.26},    '4': {'name': 'Tampere', 'fmisid': '101118', 'lat': 61.41, 'lon': 23.60},    '5': {'name': 'Jyväskylä', 'fmisid': '101339', 'lat': 62.40, 'lon': 25.67},    '6': {'name': 'Oulu', 'fmisid': '101799', 'lat': 65.03, 'lon': 25.47},    '7': {'name': 'Rovaniemi', 'fmisid': '101917', 'lat': 66.56, 'lon': 25.83},    '8': {'name': 'Sample Data', 'fmisid': 'SAMPLE', 'lat': 60.81, 'lon': 23.50}}selected_station = stations[station_choice]USE_SAMPLE_DATA = (station_choice == '8' or not FMI_API_KEY)print(f"\n✅ Valittu asema / Selected: {selected_station['name']}")print(f"   Sijainti / Location: {selected_station['lat']:.2f}°N, {selected_station['lon']:.2f}°E")# Time period (FORM)print("\n📅 AIKAJAKSO / TIME PERIOD:")print("  [1] Last 30 days / Viimeiset 30 päivää")print("  [2] Current year / Kuluva vuosi")print("  [3] Growing season 2024 (May-Sep)")print("  [4] Custom period / Oma aikajakso")period_choice = input("Valinta / Choice (1-4): ").strip() or '1'if period_choice == '1':    end_date = datetime.now()    start_date = end_date - timedelta(days=30)elif period_choice == '2':    start_date = datetime(datetime.now().year, 1, 1)    end_date = datetime.now()elif period_choice == '3':    start_date = datetime(2024, 5, 1)    end_date = datetime(2024, 9, 30)else:    start_str = input("Alkupäivä / Start date (YYYY-MM-DD): ") or "2024-05-01"    end_str = input("Loppupäivä / End date (YYYY-MM-DD): ") or "2024-09-30"    start_date = datetime.strptime(start_str, "%Y-%m-%d")    end_date = datetime.strptime(end_str, "%Y-%m-%d")print(f"\n✅ Aikajakso / Period: {start_date.date()} - {end_date.date()}")print(f"   Päiviä / Days: {(end_date - start_date).days}")# Analysis focusprint("\n🔬 ANALYYSIN PAINOPISTE / ANALYSIS FOCUS:")print("  [1] Growing Season Analysis / Kasvukausi-analyysi")print("  [2] Frost Risk / Hallariski")print("  [3] Precipitation / Sadanta")print("  [4] Temperature Trends / Lämpötilatrendit")focus_choice = input("Valinta / Choice (1-4): ").strip() or '1'print("\n✅ Asetukset tallennettu / Settings saved!")

## 🔧 Vaihe 3: Apufunktiot / Step 3: Helper Functions


In [ ]:
# ============================================================================# Helper Functions for FMI Data and Analysis# ============================================================================def generate_sample_finnish_weather(days=90, start_date=None):    """    Generate sample Finnish weather data for testing.    Luo esimerkkidataa Suomen säästä testausta varten.        Args:        days: Number of days to generate        start_date: Start date (default: 90 days ago)        Returns:        DataFrame with Finnish weather pattern    """    if start_date is None:        start_date = datetime.now() - timedelta(days=days)        dates = pd.date_range(start=start_date, periods=days, freq='D')    day_of_year = np.array([d.timetuple().tm_yday for d in dates])        # Realistic Finnish temperature pattern    # Cold winter, mild summer    base_temp = -5 + 20 * np.sin((day_of_year - 100) * 2 * np.pi / 365)    temp_max = base_temp + 5 + np.random.normal(0, 2, days)    temp_min = base_temp - 5 + np.random.normal(0, 2, days)    temp_avg = (temp_max + temp_min) / 2        # Precipitation (more in autumn)    precip_base = 1 + 2 * np.sin((day_of_year - 250) * 2 * np.pi / 365)    precipitation = np.maximum(0, precip_base + np.random.gamma(2, 1, days))        # Snow (winter months)    snow_depth = np.where(        (day_of_year < 120) | (day_of_year > 300),        np.maximum(0, 20 + 30 * np.sin((day_of_year - 20) * np.pi / 100) + np.random.normal(0, 10, days)),        0    )        # Sunshine hours (very short in winter)    max_sun = 0.5 + 18 * np.sin((day_of_year - 80) * 2 * np.pi / 365)    sunshine_hours = np.maximum(0, max_sun + np.random.normal(0, 2, days))        df = pd.DataFrame({        'date': dates,        'temp_avg': temp_avg,        'temp_max': temp_max,        'temp_min': temp_min,        'precipitation_mm': precipitation,        'snow_depth_cm': snow_depth,        'sunshine_hours': sunshine_hours,        'station': selected_station['name']    })        return dfdef calculate_gdd(temp_avg, base_temp=5.0):    """    Calculate Growing Degree Days (GDD).    Laskee kasvuastepäivät.        GDD = max(0, T_avg - T_base)        Args:        temp_avg: Average temperature        base_temp: Base temperature (default 5°C for most crops)        Returns:        Growing degree days    """    return np.maximum(0, temp_avg - base_temp)def frost_risk_analysis(temp_min):    """    Analyze frost risk from minimum temperatures.    Analysoi hallariskin minimilämpötiloista.        Args:        temp_min: Minimum temperatures        Returns:        Dict with frost statistics    """    frost_days = (temp_min < 0).sum()    severe_frost = (temp_min < -5).sum()    first_frost = None    last_frost = None        frost_dates = temp_min[temp_min < 0].index    if len(frost_dates) > 0:        first_frost = frost_dates[0]        last_frost = frost_dates[-1]        return {        'frost_days': frost_days,        'severe_frost_days': severe_frost,        'first_frost': first_frost,        'last_frost': last_frost    }def growing_season_length(temp_avg, threshold=5.0):    """    Calculate growing season length.    Laskee kasvukauden pituuden.        Growing season: period with average temp > threshold for 5+ consecutive days        Args:        temp_avg: Average temperatures (Series with date index)        threshold: Temperature threshold (default 5°C)        Returns:        Dict with season start, end, and length    """    above_threshold = temp_avg > threshold        # Find consecutive periods    season_start = None    season_end = None        # Simple approach: first 5-day period above threshold    for i in range(len(above_threshold) - 4):        if all(above_threshold.iloc[i:i+5]):            season_start = above_threshold.index[i]            break        # Last 5-day period above threshold    for i in range(len(above_threshold) - 1, 4, -1):        if all(above_threshold.iloc[i-4:i+1]):            season_end = above_threshold.index[i]            break        if season_start and season_end:        length = (season_end - season_start).days    else:        length = 0        return {        'start': season_start,        'end': season_end,        'length_days': length    }print("✅ Apufunktiot ladattu / Helper functions loaded")

## 📡 Vaihe 4: Datan haku / Step 4: Data Fetching

Haetaan säädata FMI:stä tai luodaan esimerkkidata.


In [ ]:
# ============================================================================# Fetch Weather Data# ============================================================================if USE_SAMPLE_DATA:    print("�� Käytetään esimerkkidataa / Using sample data...")    weather_data = generate_sample_finnish_weather(        days=(end_date - start_date).days,        start_date=start_date    )    print(f"✅ Luotu {len(weather_data)} päivän data / Generated {len(weather_data)} days of data")    else:    print("📡 Haetaan FMI-dataa / Fetching FMI data...")    print("⚠️ Note: Real FMI API integration requires additional XML parsing")    print("   For production use, implement fmi_wfs_request() function")    print("   API docs: https://en.ilmatieteenlaitos.fi/open-data-manual")        # Placeholder for real FMI API call    # In production, this would use FMI's WFS (Web Feature Service)    weather_data = generate_sample_finnish_weather(        days=(end_date - start_date).days,        start_date=start_date    )    weather_data['station'] = selected_station['name']    print(f"✅ Sample data generated for demonstration")# Set date as indexweather_data.set_index('date', inplace=True)# Display previewdisplay(Markdown("### 🔍 Datan esikatselu / Data Preview"))display(weather_data.head(10))# Summary statisticsdisplay(Markdown("### 📊 Yhteenveto / Summary Statistics"))display(weather_data.describe())print(f"\n📅 Aikajakso / Period: {weather_data.index[0].date()} - {weather_data.index[-1].date()}")print(f"📏 Datapisteitä / Data points: {len(weather_data)}")

## 🔬 Vaihe 5: Analyysi / Step 5: Analysis

Analysoidaan säädata puutarhatuotannon näkökulmasta.


In [ ]:
# ============================================================================# Weather Analysis for Horticulture# ============================================================================# Calculate Growing Degree Daysweather_data['gdd'] = calculate_gdd(weather_data['temp_avg'], base_temp=5.0)weather_data['gdd_cumulative'] = weather_data['gdd'].cumsum()# Frost analysisfrost_stats = frost_risk_analysis(weather_data['temp_min'])# Growing seasonseason_stats = growing_season_length(weather_data['temp_avg'], threshold=5.0)# Precipitation totalstotal_precip = weather_data['precipitation_mm'].sum()avg_precip = weather_data['precipitation_mm'].mean()# Display resultsdisplay(Markdown(f"""## 📈 Analyysin tulokset / Analysis Results### 🌡️ Lämpötila / Temperature- Keskiarvo / Average: **{weather_data['temp_avg'].mean():.1f}°C**- Maksimi / Maximum: **{weather_data['temp_max'].max():.1f}°C**- Minimi / Minimum: **{weather_data['temp_min'].min():.1f}°C**### 🌱 Kasvukausi / Growing Season- Alkaa / Starts: **{season_stats['start'].date() if season_stats['start'] else 'N/A'}**- Päättyy / Ends: **{season_stats['end'].date() if season_stats['end'] else 'N/A'}**- Pituus / Length: **{season_stats['length_days']} päivää / days**- Kasvuastepäivät / GDD: **{weather_data['gdd_cumulative'].max():.0f}**### ❄️ Halla / Frost- Hallapäiviä / Frost days (< 0°C): **{frost_stats['frost_days']}**- Ankaria hallapäiviä / Severe frost (< -5°C): **{frost_stats['severe_frost_days']}**- Ensimmäinen halla / First frost: **{frost_stats['first_frost'].date() if frost_stats['first_frost'] else 'N/A'}**- Viimeinen halla / Last frost: **{frost_stats['last_frost'].date() if frost_stats['last_frost'] else 'N/A'}**### �� Sadanta / Precipitation- Yhteensä / Total: **{total_precip:.1f} mm**- Keskiarvo/päivä / Average/day: **{avg_precip:.1f} mm**- Sateisia päiviä / Rainy days (>1mm): **{(weather_data['precipitation_mm'] > 1).sum()}**"""))print("\n✅ Analyysi valmis / Analysis complete")

## 📊 Vaihe 6: Visualisointi / Step 6: Visualization


In [ ]:
# ============================================================================# Visualizations# ============================================================================fig, axes = plt.subplots(3, 2, figsize=(16, 14))# 1. Temperature trendsax = axes[0, 0]ax.plot(weather_data.index, weather_data['temp_max'], 'r-', label='Max', alpha=0.7)ax.plot(weather_data.index, weather_data['temp_avg'], 'g-', label='Avg', linewidth=2)ax.plot(weather_data.index, weather_data['temp_min'], 'b-', label='Min', alpha=0.7)ax.axhline(y=0, color='black', linestyle='--', alpha=0.5, label='0°C')ax.set_title('Lämpötilat / Temperatures', fontweight='bold', fontsize=12)ax.set_ylabel('°C')ax.legend()ax.grid(True, alpha=0.3)# 2. Growing Degree Daysax = axes[0, 1]ax.plot(weather_data.index, weather_data['gdd_cumulative'], 'g-', linewidth=2)ax.set_title('Kumulatiiviset kasvuastepäivät / Cumulative GDD', fontweight='bold', fontsize=12)ax.set_ylabel('GDD (base 5°C)')ax.grid(True, alpha=0.3)# 3. Precipitationax = axes[1, 0]ax.bar(weather_data.index, weather_data['precipitation_mm'], alpha=0.7, color='blue')ax.set_title('Sadanta / Precipitation', fontweight='bold', fontsize=12)ax.set_ylabel('mm')ax.grid(True, alpha=0.3)# 4. Sunshine hoursax = axes[1, 1]ax.plot(weather_data.index, weather_data['sunshine_hours'], 'orange', linewidth=2)ax.set_title('Auringonpaiste / Sunshine Hours', fontweight='bold', fontsize=12)ax.set_ylabel('Tuntia / Hours')ax.grid(True, alpha=0.3)# 5. Snow depthax = axes[2, 0]ax.fill_between(weather_data.index, weather_data['snow_depth_cm'], alpha=0.6, color='lightblue')ax.set_title('Lumensyvyys / Snow Depth', fontweight='bold', fontsize=12)ax.set_ylabel('cm')ax.grid(True, alpha=0.3)# 6. Temperature distributionax = axes[2, 1]ax.hist(weather_data['temp_avg'], bins=30, alpha=0.7, color='green', edgecolor='black')ax.axvline(x=0, color='blue', linestyle='--', label='0°C')ax.axvline(x=5, color='green', linestyle='--', label='5°C (GDD base)')ax.set_title('Lämpötilajakauma / Temperature Distribution', fontweight='bold', fontsize=12)ax.set_xlabel('°C')ax.set_ylabel('Päiviä / Days')ax.legend()ax.grid(True, alpha=0.3)plt.tight_layout()plt.show()print("✅ Visualisoinnit valmiit / Visualizations complete")

## 💡 Vaihe 7: Suositukset / Step 7: Recommendations


In [ ]:
# ============================================================================# Horticultural Recommendations# ============================================================================# Generate recommendations based on analysisrecommendations = []# Planting recommendations based on frostif frost_stats['last_frost']:    safe_planting = frost_stats['last_frost'] + timedelta(days=14)    recommendations.append(f"🌱 Turvallinen kylvöaika / Safe planting after: **{safe_planting.date()}** (2 weeks after last frost)")# GDD-based crop suggestionstotal_gdd = weather_data['gdd_cumulative'].max()if total_gdd > 1500:    recommendations.append("🌾 GDD >1500: Sopii / Suitable for warm-season crops (tomatoes, peppers)")elif total_gdd > 1000:    recommendations.append("🥬 GDD 1000-1500: Sopii / Suitable for cool-season crops (lettuce, cabbage)")else:    recommendations.append("🥶 GDD <1000: Rajoitettu kausi / Limited season, choose quick-maturing varieties")# Irrigation needsif total_precip < 300:    recommendations.append("💧 Sadanta <300mm: Kastelu tarpeen / Irrigation needed")elif total_precip > 600:    recommendations.append("💧 Sadanta >600mm: Hyvä kosteus, seuraa kuivatusta / Good moisture, monitor drainage")# Frost protectionif frost_stats['frost_days'] > 0:    recommendations.append(f"❄️ {frost_stats['frost_days']} hallapäivää / frost days: Suunnittele suojaus / Plan frost protection")display(Markdown(f"""## 💡 Puutarhasuositukset / Horticultural Recommendations### Asema / Station: {selected_station['name']}### Aikajakso / Period: {start_date.date()} - {end_date.date()}{chr(10).join(['- ' + r for r in recommendations])}### 📅 Tyypillinen kasvukausi Suomessa / Typical Growing Season in Finland#### Etelä-Suomi / Southern Finland- Kylvö sisällä / Indoor sowing: March-April- Kylvö ulkona / Outdoor sowing: May (after last frost)- Istutus / Transplanting: Late May - Early June- Sadonkorjuu / Harvest: July-September#### Keski-Suomi / Central Finland- Kylvö sisällä / Indoor sowing: April- Kylvö ulkona / Outdoor sowing: Late May - Early June- Sadonkorjuu / Harvest: August-September#### Pohjois-Suomi / Northern Finland- Lyhyt kasvukausi / Short growing season: June-August- Valitse nopeat lajikkeet / Choose quick-maturing varieties- Kasvihuoneviljely suositeltavaa / Greenhouse recommended### 🌡️ Kasvuastepäivät eri kasveille / GDD Requirements for Common Crops| Kasvi / Crop | GDD | Huomiot / Notes ||--------------|-----|-----------------|| Salaatti / Lettuce | 400-800 | Viileä kausi / Cool season || Peruna / Potato | 1000-1400 | Suosittu Suomessa / Popular in Finland || Tomaatti / Tomato | 1200-2000 | Kasvihuone / Greenhouse recommended || Porkkana / Carrot | 900-1200 | Kestää viileää / Cold tolerant || Kurkku / Cucumber | 1000-1500 | Kasvihuone / Greenhouse best |"""))print("\n✅ Suositukset luotu / Recommendations generated")

## 📚 Vaihe 8: Vienti ja lähteet / Step 8: Export & Citations


In [ ]:
# ============================================================================# Export and Documentation# ============================================================================# Export dataexport_filename = f"finnish_weather_{selected_station['name'].replace(' ', '_')}_{start_date.date()}_to_{end_date.date()}.csv"weather_data.to_csv(export_filename)print(f"💾 Data viety / Data exported: {export_filename}")display(Markdown(f"""## 📖 Lähteet ja viittaukset / Sources and Citations### Tietolähteet / Data Sources**Ilmatieteen laitos (FMI) / Finnish Meteorological Institute**- URL: https://www.ilmatieteenlaitos.fi- API: https://en.ilmatieteenlaitos.fi/open-data- License: Creative Commons Attribution 4.0 (CC BY 4.0)- Citation: "Ilmatieteen laitos, {datetime.now().year}. Weather observations from {selected_station['name']} station."### Viittaus tähän analyysiin / Citation for This Analysis> Botanical Colabs (2025). Finnish Horticultural Weather Analysis.> Based on FMI Open Data. > https://github.com/outobecca/botanical-colabs### Lisätietoja / Additional Resources**Puutarhatutkimus / Horticultural Research:**- Luke (Luonnonvarakeskus): https://www.luke.fi- Jokioinen Research Station: https://www.luke.fi/toimipaikat/jokioinen**Kasvukausi-info / Growing Season Information:**- Puutarhaliiton kasvukausikartat / Finnish Garden Association growing zone maps- Pohjoisen puutarhanviljelyn keskus / Northern Horticulture Centre**Ilmasto / Climate:**- FMI Climate Service: https://www.ilmatieteenlaitos.fi/ilmastopalvelut- Finnish climate statistics and trends### ⚖️ Lisenssi / LicenseThis notebook: MIT LicenseFMI Data: CC BY 4.0Sample/simulated data: Public domain### ⚠️ Vastuuvapauslauseke / Disclaimer- This analysis is for educational and planning purposes- Weather data may have gaps or inaccuracies- Always verify critical decisions with current forecasts- Consult local agricultural extension services for specific advice**Tämä analyysi on tarkoitettu koulutus- ja suunnittelutarkoituksiin. Tarkista aina kriittiset päätökset ajankohtaisista ennusteista.**"""))print("\n✅ Analyysi valmis! / Analysis complete!")print(f"📊 Yhteensä {len(weather_data)} päivän data analysoitu")print(f"🇫🇮 Asema: {selected_station['name']}")